This example is for Pydantic output parser.

In [42]:
from typing import List

from langchain_ollama import OllamaLLM
from pydantic import BaseModel, Field, validator
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
import json

In [44]:
llm = OllamaLLM(model="deepseek-r1:7b", temperature=0.5)


Below is method to define Pydantic call.

In [45]:
class Ticket(BaseModel):
    date: str = Field(description="Show Date")
    time: str = Field(description="Show Time")
    theater: str = Field(description="Theater Name")
    count: int = Field(description="Number of Tickets")
    movie: str = Field(description="Movie Name")

In [46]:
parser = PydanticOutputParser(pydantic_object=Ticket)

Define Template for the Prompt

In [47]:
ticket_template = '''
  Book us a movie ticket for two this Friday at 6 PM.
  Choose any theater, it doesn't matter. Send confirmation by email.
  Our preferred movie is: "{query}"
  Format instructions: {format_instructions}
  Please respond ONLY with valid JSON output matching the Ticket schema.
'''

Populate the Prompt Template

In [48]:
prompt = PromptTemplate(
    template=ticket_template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

Construct the prompt

In [49]:
input = prompt.format_prompt(query="Inception")
print("Prompt to LLM:\n", input.to_string())

Prompt to LLM:
 
  Book us a movie ticket for two this Friday at 6 PM.
  Choose any theater, it doesn't matter. Send confirmation by email.
  Our preferred movie is: "Inception"
  Format instructions: The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"date": {"description": "Show Date", "title": "Date", "type": "string"}, "time": {"description": "Show Time", "title": "Time", "type": "string"}, "theater": {"description": "Theater Name", "title": "Theater", "type": "string"}, "count": {"description": "Number of Tickets", "title": "Count", "type": "integer"}, "movie": {"descripti

In [50]:
output = llm.invoke(input.to_string())
print("\nLLM Output:\n", output)


LLM Output:
 ```json
{
  "date": "2023-10-06",
  "time": "18:00",
  "theater": "Any Theater",
  "count": 2,
  "movie": "Inception"
}
```


In [51]:
reservation = parser.parse(output)
reservation

Ticket(date='2023-10-06', time='18:00', theater='Any Theater', count=2, movie='Inception')

In [52]:
type(reservation)

__main__.Ticket